In [51]:
# Provides ways to work with large multidimensional arrays
import numpy as np 
# Allows for further data manipulation and analysis
import pandas as pd 
import matplotlib.pyplot as plt # Plotting
import matplotlib.dates as mdates # Styling dates
%matplotlib inline

# pip install numpy
# conda install -c anaconda pandas
# conda install -c conda-forge matplotlib

import datetime as dt # For defining dates

import time

# In Powershell Prompt : conda install -c conda-forge multitasking
# pip install -i https://pypi.anaconda.org/ranaroussi/simple yfinance

import yfinance as yf

# To show all your output File -> Preferences -> Settings Search for Notebook
# Notebook Output Text Line Limit and set to 100

# Used for file handling like deleting files
import os

# conda install -c conda-forge cufflinks-py
# conda install -c plotly plotly
import cufflinks as cf
import plotly.express as px
import plotly.graph_objects as go

# Make Plotly work in your Jupyter Notebook
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
# Use Plotly locally
cf.go_offline()

from plotly.subplots import make_subplots

# New Imports
# Used to get data from a directory
import os
from os import listdir
from os.path import isfile, join

import warnings
warnings.simplefilter("ignore")

In [52]:
PATH = "C:/Users/Taylor/Documents/Documents/Invest_DS/StockTickerData/nasdaq/"

In [53]:
files = [x for x in listdir(PATH) if isfile(join(PATH, x))]
tickers = [os.path.splitext(x)[0] for x in files]
tickers
# tickers.remove('.ds_Store') MacOS Only
tickers.sort()
len(tickers)

10

In [54]:
def get_stock_df_from_csv(ticker):
    try:
        df = pd.read_csv(PATH + ticker + '.csv', index_col=0)
    except FileNotFoundError:
        print("File Doesn't Exist")
    else:
        return df    

In [55]:
def add_daily_return_to_df(df):
    df['daily_return'] = (df['Close'] - df['Close'].shift(1)) / df['Close'].shift(1)
    # df['daily_return'] = (df['Close'] / df['Close'].shift(1)) - 1
    return df

In [56]:
def add_cum_return_to_df(df):
    df['cum_return'] = (1 + df['daily_return']).cumprod()
    return df

In [57]:
def add_moving_average(df, day):
    df['moving_average'] = df['Close'].rolling(window=day).mean()
    return df

In [58]:
def add_std_deviation(df, day):
    df['std_deviation'] = df['Close'].rolling(window=day).std()
    return df

In [59]:
def add_bollinger_bands(df):
    df['upper_band'] = df['moving_average'] + 1.96 * df['std_deviation']
    df['lower_band'] = df['moving_average'] - 1.96 * df['std_deviation']
    return df

In [60]:
def add_ichimoku(df):
    hi_val_1 = df['High'].rolling(window=9).max()
    low_val_1 = df['Low'].rolling(window=9).min()
    df['tenkan-sen'] = hi_val_1 + low_val_1 / 2 

    hi_val_2 = df['High'].rolling(window=26).max()
    low_val_2 = df['Low'].rolling(window=26).min()
    df['kijun-sen'] = hi_val_2 + low_val_2 / 2 

    df['senkou_span_a'] = ((df['tenkan-sen'] + df['kijun-sen']) / 2 ).shift(26)

    hi_val_3 = df['High'].rolling(window=52).max()
    low_val_3 = df['Low'].rolling(window=52).min()
    df['senkou_span_b'] = ((df['tenkan-sen'] + df['kijun-sen']) / 2 ).rolling(window=52).mean()

    df['Chikou_span'] = df['Close'].shift(-26)

    return df

In [61]:
for x in tickers:
    try:
        print("Working on :", x)
        df = get_stock_df_from_csv(x)
        df = add_daily_return_to_df(df)
        df = add_cum_return_to_df(df)
        df = add_moving_average(df, day)
        df = add_std_deviation(df, day)
        df = add_bollinger_bands(df)
        df = add_ichimoku(df)
        df.to_csv(PATH + x + ".csv")
    except Exception as ex:
        print(ex)
    print('Saved')

Working on : AACG
Saved
Working on : AACI
Saved
Working on : AADI
Saved
Working on : AAL
Saved
Working on : AAME
Saved
Working on : AAOI
Saved
Working on : AAON
Saved
Working on : AAPL
Saved
Working on : AATC
Saved
Working on : AAWW
Saved
